In [1]:
import onnxruntime as ort

providers = ["CPUExecutionProvider"]
sess_options = ort.SessionOptions()
onnx_sess = ort.InferenceSession(
    "vits.onnx",
    sess_options=sess_options,
    providers=providers,
)

In [2]:
import numpy as np

input_ids = [0, 64, 0, 156, 0, 102, 0, 62, 0, 61, 0, 16, 0, 102, 0, 68, 0, 16, 0, 156, 0, 76, 0, 158, 0, 61, 0, 138, 0, 55, 0, 5, 0]
input_lengths = np.array([len(input_ids)], dtype=np.int64)
input_ids = np.array([input_ids], dtype=np.int64)
noise_scale = np.array([.667], dtype=np.float32)
noise_scale_w = np.array([.8], dtype=np.float32)
length_scale = np.array([1.0], dtype=np.float32)

In [3]:
outputs = onnx_sess.run(
    None,
    {
        "input": input_ids,
        "input_lengths": input_lengths,
        "noise_scale": noise_scale,
        "noise_scale_w": noise_scale_w,
        "length_scale": length_scale,
        "speaker_id": None,
    },
)

In [6]:
audio, attention = outputs

In [7]:
import IPython.display as ipd

ipd.Audio(audio[0, 0, :], rate=22050)

In [8]:
audio.shape

(1, 1, 25088)

In [10]:
attention.shape

(1, 1, 98, 33)

In [12]:
duration_s = audio.shape[-1] / 22050
duration_s

1.1377777777777778

In [13]:
durations = np.sum(attention[0][0], axis=0)
durations.shape, durations

((33,),
 array([1., 3., 3., 2., 2., 1., 2., 1., 3., 4., 1., 3., 2., 1., 3., 2., 1.,
        5., 2., 4., 3., 1., 6., 3., 5., 5., 2., 6., 5., 4., 1., 5., 6.],
       dtype=float32))

In [ ]:
# 1 acoustic unit = 256 frames = 256 frames / 22050 fps = 0,011609977324263 s